## setup

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

from collections import defaultdict

import json

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score

import itertools

import random

from skimage import measure

import pickle

import gc

In [ ]:
def read_image(path):
    return plt.imread(path)

def read_annotation_file(path):
    with open(path) as annotation_file:
        annotation_list = json.load(annotation_file)
    # Transform list of annotations into dictionary
    annotation_dict = {}
    for annotation in annotation_list:
        sequence_id = annotation['sequence_id']
        if sequence_id not in annotation_dict:
            annotation_dict[sequence_id] = {}
        annotation_dict[sequence_id][annotation['frame']] = annotation['object_coords']
    return annotation_dict

random.seed(0)

def random_different_coordinates(coords, size_x, size_y, pad):
    """ Returns a random set of coordinates that is different from the provided coordinates, 
    within the specified bounds.
    The pad parameter avoids coordinates near the bounds."""
    good = False
    while not good:
        good = True
        c1 = random.randint(pad + 1, size_x - (pad + 1))
        c2 = random.randint(pad + 1, size_y -( pad + 1))
        for c in coords:
            if c1 == c[0] and c2 == c[1]:
                good = False
                break
    return (c1,c2)

def extract_neighborhood(x, y, arr, radius):
    """ Returns a 1-d array of the values within a radius of the x,y coordinates given """
    return arr[(x - radius) : (x + radius + 1), (y - radius) : (y + radius + 1)].ravel()

def check_coordinate_validity(x, y, size_x, size_y, pad):
    """ Check if a coordinate is not too close to the image edge """
    return x >= pad and y >= pad and x + pad < size_x and y + pad < size_y

def generate_labeled_data(image_path, annotation, nb_false, radius):
    """ For one frame and one annotation array, returns a list of labels 
    (1 for true object and 0 for false) and the corresponding features as an array.
    nb_false controls the number of false samples
    radius defines the size of the sliding window (e.g. radius of 1 gives a 3x3 window)"""
    features,labels = [],[]
    im_array = read_image(image_path)
    # True samples
    for obj in annotation:
        obj = [int(x + .5) for x in obj] #Project the floating coordinate values onto integer pixel coordinates.
        # For some reason the order of coordinates is inverted in the annotation files
        if check_coordinate_validity(obj[1],obj[0],im_array.shape[0],im_array.shape[1],radius):
            features.append(extract_neighborhood(obj[1],obj[0],im_array,radius))
            labels.append(1)
    # False samples
    for i in range(nb_false):
        c = random_different_coordinates(annotation,im_array.shape[1],im_array.shape[0],radius)
        features.append(extract_neighborhood(c[1],c[0],im_array,radius))
        labels.append(0)
    return np.array(labels),np.stack(features,axis=1)

def generate_labeled_set(annotation_array, path, sequence_id_list, radius, nb_false):
    # Generate labeled data for a list of sequences in a given path
    labels,features = [],[]
    for seq_id in sequence_id_list:
        for frame_id in range(1,6):
            d = generate_labeled_data(f"{path}{seq_id}/{frame_id}.png",
                                    annotation_array[seq_id][frame_id],
                                    nb_false,
                                    radius)
            labels.append(d[0])
            features.append(d[1])
    return np.concatenate(labels,axis=0), np.transpose(np.concatenate(features,axis=1))

def classify_image(im, model, radius):
    n_features=(2*radius+1)**2 #Total number of pixels in the neighborhood
    feat_array=np.zeros((im.shape[0],im.shape[1],n_features))
    for x in range(radius+1,im.shape[0]-(radius+1)):
        for y in range(radius+1,im.shape[1]-(radius+1)):
            feat_array[x,y,:]=extract_neighborhood(x,y,im,radius)
    all_pixels=feat_array.reshape(im.shape[0]*im.shape[1],n_features)
    pred_pixels=model.predict(all_pixels).astype(np.bool_)
    pred_image=pred_pixels.reshape(im.shape[0],im.shape[1])
    return pred_image

def extract_centroids(pred, bg):
    conn_comp=measure.label(pred, background=bg)
    object_dict=defaultdict(list) #Keys are the indices of the connected components and values are arrrays of their pixel coordinates 
    for (x,y),label in np.ndenumerate(conn_comp):
            if label != bg:
                object_dict[label].append([x,y])
    # Mean coordinate vector for each object, except the "0" label which is the background
    centroids={label: np.mean(np.stack(coords),axis=0) for label,coords in object_dict.items()}
    object_sizes={label: len(coords) for label,coords in object_dict.items()}
    return centroids, object_sizes

def filter_large_objects(centroids,object_sizes, max_size):
    small_centroids={}
    for label,coords in centroids.items():
            if object_sizes[label] <= max_size:
                small_centroids[label]=coords
    return small_centroids

def predict_objects(sequence_id, frame_id, model, radius, max_size):
    test_image = plt.imread(f"../input/spotgeo/test/test/{sequence_id}/{frame_id}.png")
    test_pred=classify_image(test_image, model, radius)
    test_centroids, test_sizes = extract_centroids(test_pred, 0)
    test_centroids = filter_large_objects(test_centroids, test_sizes, max_size)
    # Switch x and y coordinates for submission
    if len(test_centroids.values()) > 0:
        sub=np.concatenate([c[np.array([1,0])].reshape((1,2)) for c in test_centroids.values()])
        #np array converted to list for json seralization, truncated to the first 30 elements
        return sub.tolist()[0:30]
    else:
        return []

## data preparation

In [ ]:
random_state = 0

In [ ]:
train_annotation = read_annotation_file('../input/spotgeo/train_anno.json')

In [ ]:
len(train_annotation)

In [ ]:
%%time

radius = 3
train_labels, train_features = generate_labeled_set(train_annotation, '../input/spotgeo/train/train/', range(1,1001), radius, 10)

print(train_labels.shape)
print(train_features.shape)

In [ ]:
%%time

validation_labels, validation_features = generate_labeled_set(train_annotation, '../input/spotgeo/train/train/', range(1001,1281), radius, 10)

print(validation_labels.shape)
print(validation_features.shape)

## train random forest

In [ ]:
%%time

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state = random_state, n_jobs = -1)

model.fit(train_features, train_labels)

In [ ]:
pred_labels = model.predict(validation_features)

print(classification_report(pred_labels, validation_labels))
print('\n')
print(confusion_matrix(pred_labels, validation_labels))
print('\n')
print("Kappa =", cohen_kappa_score(pred_labels, validation_labels))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [10,20,50,100,250,500,750,1000]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
%%time

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=10, cv=3, verbose=2, random_state=random_state, n_jobs=-1, scoring='f1')

rf_random.fit(train_features[:10000], train_labels[:10000])

In [ ]:
rf_random.best_params_

In [ ]:
model = RandomForestClassifier(n_estimators = rf_random.best_params_['n_estimators'],
                               max_features = rf_random.best_params_['max_features'],
                               max_depth = rf_random.best_params_['max_depth'],
                               min_samples_split = rf_random.best_params_['min_samples_split'],
                               min_samples_leaf = rf_random.best_params_['min_samples_leaf'],
                               bootstrap = rf_random.best_params_['bootstrap'],
                               n_jobs = -1,
                               random_state = random_state)

In [ ]:
%%time

model.fit(train_features, train_labels)

In [ ]:
pickle.dump(model, open('/kaggle/working/rf.sav', 'wb'))

In [ ]:
pred_labels = model.predict(validation_features)

print(classification_report(pred_labels, validation_labels))
print('\n')
print(confusion_matrix(pred_labels, validation_labels))
print('\n')
print("Kappa =", cohen_kappa_score(pred_labels, validation_labels))

In [ ]:
%%time

sub_list=predict_objects(1,1,model,radius,1)
print(sub_list[0:5])

## train xgb

In [ ]:
%%time

import xgboost as xgb

model = xgb.XGBClassifier(random_state = random_state, n_jobs = -1)

model.fit(train_features, train_labels)

In [ ]:
pred_labels = model.predict(validation_features)

print(classification_report(pred_labels, validation_labels))
print('\n')
print(confusion_matrix(pred_labels, validation_labels))
print('\n')
print("Kappa =", cohen_kappa_score(pred_labels, validation_labels))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [10,20,50,100,250,500,750,1000]

learning_rate = [0.1,0.05,0.01,0.005,0.001]

max_depth = [3,4,5,6,7]

colsample_bytree = [0.7,0.8,0.9,1]

subsample = [0.7,0.8,0.9,1]

gamma = [0,1,5]

random_grid = {'n_estimators': n_estimators,
               'learning_rate': learning_rate,
               'max_depth': max_depth,
               'colsample_bytree': colsample_bytree,
               'subsample': subsample,
               'gamma': gamma}

print(random_grid)

In [ ]:
%%time

import xgboost as xgb

xgb = xgb.XGBClassifier()

xgb_random = RandomizedSearchCV(estimator = xgb, param_distributions = random_grid, n_iter = 10, cv = 3, verbose = 2, random_state = random_state, n_jobs = -1, scoring = 'f1')

xgb_random.fit(train_features[:10000], train_labels[:10000])

In [ ]:
xgb_random.best_params_

In [ ]:
import xgboost as xgb

model = xgb.XGBClassifier(n_estimators = xgb_random.best_params_['n_estimators'],
                          learning_rate = xgb_random.best_params_['learning_rate'],
                          max_depth = xgb_random.best_params_['max_depth'],
                          colsample_bytree = xgb_random.best_params_['colsample_bytree'],
                          subsample = xgb_random.best_params_['subsample'],
                          gamma = xgb_random.best_params_['gamma'],
                          n_jobs = -1,
                          random_state = random_state)

In [ ]:
%%time

model.fit(train_features, train_labels)

In [ ]:
pickle.dump(model, open('/kaggle/working/xgb.sav', 'wb'))

In [ ]:
pred_labels = model.predict(validation_features)

print(classification_report(pred_labels, validation_labels))
print('\n')
print(confusion_matrix(pred_labels, validation_labels))
print('\n')
print("Kappa =", cohen_kappa_score(pred_labels, validation_labels))

In [ ]:
%%time

sub_list=predict_objects(1,1,model,radius,1)
print(sub_list[0:5])

## train 5-layer neural net (sklearn)

In [ ]:
%%time

from sklearn.neural_network import MLPClassifier

dim = train_features.shape[1]

model = MLPClassifier(hidden_layer_sizes=(dim,dim,dim), activation = 'relu', solver='adam', random_state=random_state)

model.fit(train_features, train_labels)

In [ ]:
pickle.dump(model, open('/kaggle/working/nn5.sav', 'wb'))

In [ ]:
pred_labels = model.predict(validation_features)

print(classification_report(pred_labels, validation_labels))
print('\n')
print(confusion_matrix(pred_labels, validation_labels))
print('\n')
print("Kappa =", cohen_kappa_score(pred_labels, validation_labels))

In [ ]:
%%time

sub_list=predict_objects(1,1,model,radius,1)
print(sub_list[0:5])

## train 5-layer neural net (keras)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import backend as K

dim = train_features.shape[1]

model = Sequential()
model.add(Dense(dim, input_dim=dim, activation='relu'))
model.add(Dropout(0.2, seed=random_state))
model.add(Dense(dim, activation='relu'))
model.add(Dropout(0.2, seed=random_state))
model.add(Dense(dim, activation='relu'))
model.add(Dropout(0.2, seed=random_state))
model.add(Dense(dim, activation='relu'))
model.add(Dropout(0.2, seed=random_state))
model.add(Dense(1, activation='linear'))

In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
model.summary()

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [ ]:
%%time

epochs = 100
batch_size = 32

history = model.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(validation_features, validation_labels))

gc.collect()

In [ ]:
model.save('/kaggle/working/nn5')

In [ ]:
plt.plot(history.history['f1'])
plt.title('F1')
plt.ylabel('F1')
plt.xlabel('Epoch')
plt.show()

In [ ]:
pred_labels = model.predict_classes(validation_features)

print(classification_report(pred_labels, validation_labels))
print('\n')
print(confusion_matrix(pred_labels, validation_labels))
print('\n')
print("Kappa =", cohen_kappa_score(pred_labels, validation_labels))

In [ ]:
def classify_image(im, model, radius):
    n_features=(2*radius+1)**2 #Total number of pixels in the neighborhood
    feat_array=np.zeros((im.shape[0],im.shape[1],n_features))
    for x in range(radius+1,im.shape[0]-(radius+1)):
        for y in range(radius+1,im.shape[1]-(radius+1)):
            feat_array[x,y,:]=extract_neighborhood(x,y,im,radius)
    all_pixels=feat_array.reshape(im.shape[0]*im.shape[1],n_features)
    pred_pixels=model.predict_classes(all_pixels).astype(np.bool_)
    pred_image=pred_pixels.reshape(im.shape[0],im.shape[1])
    return pred_image

In [ ]:
%%time

sub_list=predict_objects(1,1,model,radius,1)
print(sub_list[0:5])